# Imports

In [1]:
import torch
!pip install torch-geometric torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 35.3 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=e6716d82963cdba336c508c3f97964d3d961733d0da04a0dff63feabe14def42
  Stored in directory: /root/.cache/pip/wheels/31/b2/8c/9b4bb72a4384eabd1ffeab2b7ead692c9165e35711f8a9dc72
Successfully built torch-geometric


In [2]:
import multiprocessing as mp
import os
import pickle
import warnings
import torch
from functools import partial
from itertools import combinations
import numpy as np
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity as cos_sim
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import BallTree
from sklearn.preprocessing import StandardScaler, normalize
from tqdm import tqdm

from scipy.linalg import orthogonal_procrustes
from torch_geometric.datasets import Planetoid

In [3]:
# use google drive for saving and loading information
from google.colab import drive
import pickle
import os

drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/L45_project/'

Mounted at /content/drive


# Cora Data Set Stats

In [4]:
cora_dataset = Planetoid("/tmp/cora", name="cora", split="full")
cora_data = cora_dataset[0]

Processing...
Done!


In [21]:
# Create a random list of indices with 10 items from each class for 2nd cossim
cora_labels = cora_dataset[0].y.detach().numpy()
cora_num_classes = 7
num_samples = 10
cora_indices = []
for i in range(cora_num_classes):
  class_i = np.random.choice(np.where(cora_labels == i)[0], size=num_samples, replace=False)
  print(class_i)
  cora_indices += class_i.tolist()


[ 858  260 2219 1925  753 2419 2544 2274 2089 2449]
[1780 1784 2324  831  958  556 2488 2327 2319 1505]
[2627 2656  558 2279 2409  882  389 1103 2167 1743]
[1217  649 1933  267 1442 1502 2700 2042  207  620]
[2122 1890 2376 2351 1795  749  258 1087 2432 1012]
[ 720  573 2105 2101  474 2428  272  735  739 2188]
[1914  908 1504 1402 1396 2468  724  106  778  108]


# Metric Calculations

In [6]:
# Adapted from https://github.com/SGDE2020/embedding_stability/blob/master/lib/tools/comparison.py

class Comparison:

    def __init__(self, emb_dir, embeddings, num_nodes, file_prefix):
        """
        emb_dir: str, path where embeddings are saved
        embeddings: list, list of strings that specify the embedding files
        """
        self.dir = emb_dir
        self.embeddings = embeddings
        self.pairs = self._combinations(embeddings)
        self.num_vertices = num_nodes

        # use file names without numbering to mark result files
        self.file_prefix = file_prefix

    def _combinations(self, emb_list):
        """ Computes all different comparison pairs of a list of embeddings """
        return [pair for pair in combinations(emb_list, 2)]

    def _analyse_jaccard(self, queries, nodes, k, pair):
        """This function is called in the multiprocessing of jaccard score"""
        indices_0 = np.asarray(queries[pair[0]])
        indices_1 = np.asarray(queries[pair[1]])
        nodes = np.asarray(nodes)
        jaccard_score = {}
        for i in range(len(nodes)):
            jaccard_score[nodes[i]] = \
                len(np.intersect1d(indices_0[i, 1:(k + 1)], indices_1[i, 1:(k + 1)], assume_unique=True)) / \
                len(np.union1d(indices_0[i, 1:(k + 1)], indices_1[i, 1:(k + 1)]))
        return list(jaccard_score.values())

    def _analyse_second_cossim(self, queries, normed_embs, nodes, k, pair):
        """
        This function is called in the multiprocessing of the second order cosine similarity.
        """

        # Convert the indices of nearest neighbors back into numpy
        indices_0 = np.asarray(queries[pair[0]])
        indices_1 = np.asarray(queries[pair[1]])

        # Convert the embeddings and nodes back into numpy
        norm_emb_0 = np.asarray(normed_embs[pair[0]])
        norm_emb_1 = np.asarray(normed_embs[pair[1]])
        nodes = np.asarray(nodes)

        # Compute the second order cosine similarity
        pair_results = []
        for i in range(len(nodes)):
            # Build the set of nearest neighbors w.r.t. both embeddings
            # Use indices from 1 to k+1, because the first entry will always be the node itself
            neighbors_union = np.union1d(indices_0[i, 1:(k + 1)], indices_1[i, 1:(k + 1)])

            # Vectors of cosine similarity values of nearest neighbors. There was an error in the original source code, did not use norm_emb_1 for m1
            m0 = cos_sim(norm_emb_0[neighbors_union], norm_emb_0[nodes[i]].reshape(1, -1))
            m1 = cos_sim(norm_emb_1[neighbors_union], norm_emb_1[nodes[i]].reshape(1, -1))

            # Flatten output matrix
            assert m0.shape[1] == 1 and m1.shape[1] == 1, "m0 and m1 should only have a single variable in the second dimension"
            m0 = m0.flatten()
            m1 = m1.flatten()

            # Cosine similarity between similarity vectors
            pair_results.append(float(1-cosine(m0,m1)))
        return pair_results

    def k_nearest_neighbors(self, nodes=None, append=False, samples=100, k=10, jaccard=False, load=False,
                            kload_size=100, save=True, save_path=None, num_processes=4):
        """
        Computes the k nearest neighbors to some specified nodes with respect to cosine similarity. As an intermediate
        step, it computes the 100 nearest neighbors and saves them to file. Based on these neighbors, the k-nn overlaps
        are computed.
        Args:
            nodes: list, that specifies the nodes
            append: bool, if nodes are specified, whether additional nodes will be sampled and added to nodes
            samples: int, number of nodes that will be sampled
            k: int, number of neighbors that will be used in the comparison
            jaccard: bool, whether jaccard score or overlap will be used as similarity measure
            load: bool, whether a file of computed neighbors will be used
            kload_size: Size of k in knn file to load
            save: bool, whether the results should be saved in a text file
            save_path: str, path where the file will be saved
            num_processes: number of random processes in parallelization
        Returns:
            dict, "nodes": array of nodes, "overlaps": array of overlaps, columns are values per node; or array of
            jaccard scores
        """
        # Handle the nodes input: Whether to sample nodes, use given nodes, or both.
        nodes = self._get_nodes(nodes, samples, append)

        # Load k nearest neighbors to save time? Else we will have to compute them first.
        if load:
            file_name = self.file_prefix + "_" + str(kload_size) + "nns.pickle"
            with open(os.path.join(save_path, file_name), "rb") as pickle_file:
                queries = pickle.load(pickle_file)
            assert list(queries.keys()) == self.embeddings, ("Keys of loaded queries do not match"
                                                             " with available embeddings")

        # Normalizing the embeddings to be able to use distance as a proxy for cosine similarity
        # BallTree from sklearn is used to compute the neighbors efficiently
        else:
            queries = self.nearest_neighbors_queries(nodes, k, save_path)
        # Store the naive overlaps for all pairs

        print(f"\n\n {queries.keys()} \n")

        # Use multiprocessing to speed up overlap computation.
        # Too much data is passed to the processes which makes it inefficient.
        # Possibly, it is faster to store the data as a file as an intermediate step.
        # only run if less than 100 neighbors are queried, as too large neighborhoods may cause memory issues when
        # distributing tasks
        if num_processes > 1 and k <= 100:
            with mp.Pool(num_processes) as p:
                # arguments passed in multiprocessing must be pickable
                p_queries = queries
                p_nodes = nodes.tolist()
                if jaccard:
                    multiprocess_func = partial(self._analyse_jaccard, p_queries, p_nodes, k)
                else:
                    multiprocess_func = partial(self._analyse_knn, p_queries, p_nodes, k)
                li_overlap = []
                for result in tqdm(p.imap(multiprocess_func, self.pairs), total=len(self.pairs)):
                  li_overlap.append(result)
        else:
            if jaccard:
                li_overlap = [self._analyse_jaccard(queries, nodes.tolist(), k, pair) for pair in self.pairs]
            else:
                li_overlap = [self._analyse_knn(queries, nodes.tolist(), k, pair) for pair in self.pairs]

        # Convert the result into numpy
        overlap = np.asarray(li_overlap)

        # Save the results
        if jaccard:
            nodes_suffix = "jaccard_nodes"
            scores_suffix = f"{k}nn_jaccard"
        else:
            nodes_suffix = "overlap_nodes"
            scores_suffix = f"{k}nn_overlap"
        if save is True:
            np.save(os.path.join(save_path, f"{self.file_prefix}_{nodes_suffix}"), nodes)
            np.save(os.path.join(save_path, f"{self.file_prefix}_{scores_suffix}"), overlap)
        return {"nodes": nodes, "overlaps": overlap}

    def jaccard_similarity(self, nodes=None, append=False, samples=100, k=10, load=False, kload_size=100, save=True,
                           save_path=None, num_processes=4):
        """
        Alias for k_nearest_neighbors with jaccard=True.
        See k_nearest_neighbors for detailed documentation.
        """
        return self.k_nearest_neighbors(nodes=nodes, append=append, samples=samples, k=k, jaccard=True, load=load,
                                        kload_size=kload_size, save=save, save_path=save_path,
                                        num_processes=num_processes)


    def second_order_cosine_similarity(self, nodes=None, append=False, num_samples=1000, k=10, load=True, save=False,
                                       save_path=None, num_processes=4):
        """ Computes second order cosine similarity.
        Args:
            nodes: list, that specifies the nodes
            append: bool, if nodes are specified, whether additional nodes will be sampled and added to nodes
            num_samples: int, number of nodes that will be sampled
            k: int, number of neighbors that will be used in the comparison
            load: bool, whether to load nearest neighbors from file
            save: bool, whether the results should be saved
            save_path: str, path where the file will be saved
            num_processes: number of random processes in parallelization
        Returns:
            nodes: numpy array of used nodes
            results: numpy array of similarity values of size (number of embedding pairs, number of nodes)
        """

        # Handle the nodes input: Whether to sample nodes, use given nodes, or both.
        nodes = self._get_nodes(nodes, num_samples, append)

        # Load required data: nearest neighbors, embeddings
        normed_embs = {}

        if load:
            # Load nearest neighbors from file
            file_name = self.file_prefix + "_" + str(k)+ "nns.pickle"
            with open(os.path.join(save_path, file_name), "rb") as pickle_file:
                queries = pickle.load(pickle_file)
            assert list(queries.keys()) == self.embeddings, ("Keys of loaded queries do not match with "
                                                             "available embeddings")
            for emb in tqdm(self.embeddings, desc="Loading nearest neighbor files"):
                normed_embs[emb] = normalize(self.read_embedding(os.path.join(self.dir, emb)), norm='l2', copy=False)
        else:
            queries, normed_embs = self.nearest_neighbors_queries(nodes, k, save_path, return_embeddings=True)

        # Start computation of second order cosine similarity
        # arguments passed in multiprocessing must be pickable
        p_normed_embs = dict([(key, norm_emb.tolist()) for key, norm_emb in normed_embs.items()])
        p_nodes = nodes.tolist()
        # Avoid multiprocessing on Colab, not sure if it works locally though
        if num_processes > 1 and k <= 100:
            with mp.Pool(num_processes) as p:
                li_results = p.map(partial(self._analyse_second_cossim, queries, p_normed_embs, p_nodes, k), self.pairs)
              # li_results = []
              # partial_func = partial(self._analyse_second_cossim, queries, p_normed_embs, p_nodes, k) 
              # for result in tqdm(p.imap(partial_func, self.pairs), total=len(self.pairs)):
              #   li_results.append(result)
        else:
          li_results = []
          for pair in tqdm(self.pairs, desc="Comparing embeddings"):
            li_results.append(self._analyse_second_cossim(queries, p_normed_embs, p_nodes, k, pair))

        results = np.asarray(li_results)

        if save is True:
            np.save(os.path.join(save_path, f"{self.file_prefix}_{k}nn_2nd_order_cossim"), results)

        return nodes, results

    def nearest_neighbors_queries(self, nodes, k, save_path, return_embeddings=False):
        """Uses a ball tree to compute the nearest neighbors in the embedding space"""
        queries = {}
        normed_embs = {}
        # Normalizing the embeddings to be able to use distance as a proxy for cosine similarity
        # BallTree from sklearn is used to compute the neighbors efficiently
        if return_embeddings:
            for emb in tqdm(self.embeddings, desc="Querying nearest neighbors"):
                normed_embs[emb] = normalize(self.read_embedding(os.path.join(self.dir, emb)), norm='l2', copy=False)
                ball_tree = BallTree(normed_embs[emb], leaf_size=40)
                queries[emb] = ball_tree.query(normed_embs[emb][nodes, :], k=k + 1, return_distance=False).tolist()
        else:
            for emb in tqdm(self.embeddings, desc="Querying nearest neighbors"):
                normalized_embedding = normalize(self.read_embedding(os.path.join(self.dir, emb)), norm='l2',
                                                 copy=False)
                ball_tree = BallTree(normalized_embedding, leaf_size=40)
                # Query the k+1 nearest neighbors, because a node will always be the closest neighbor to itself
                queries[emb] = ball_tree.query(normalized_embedding[nodes, :], k=k + 1, return_distance=False).tolist()

        # Save the computed neighbors to be able to skip the computation
        self.save_pickle(queries, save_path, self.file_prefix + "_" + str(k) + "nns")

        if return_embeddings:
            return queries, normed_embs
        else:
            return queries

    def sample_nodes(self, k):
        """
        Sample unique nodes of an embedding
        Args:
            k: int, number of nodes to sample
        Returns:
            numpy array of node ids of length k if k is smaller than the number of nodes available.
            Otherwise, all available nodes are returned.
        """
        vertices = np.arange(self.num_vertices)
        np.random.shuffle(vertices)
        return vertices[:min(k, self.num_vertices)]


    def cossim_analysis(self, save_path):
        """ Computes aligned cosine similarity values. Internally performs orthogonal transformation (Procrustes
        problem) between two embeddings and saves transformation matrices as well as vector of resulting errors
        """

        # Set up file naming
        results_suffix = "aligned_cossim"

        # Read the embeddings
        normed_embs = {}
        for emb in tqdm(self.embeddings, desc="Reading embeddings"):
            normed_embs[emb] = normalize(
                    self.read_embedding(os.path.join(self.dir, emb))[np.arange(self.num_vertices)], norm='l2',
                    copy=False)

        # Do the analysis
        emb_ind = -1
        results = []
        for pair in tqdm(self.pairs, desc="Comparing embeddings"):

            # only update first embedding if it does not change
            if emb_ind != pair[0]:
                emb_ind = pair[0]

            W1 = normed_embs[emb_ind]

            # transform W2 into W1 using procrustes matrix
            Q, _ = orthogonal_procrustes(normed_embs[pair[1]], normed_embs[pair[0]], check_finite=False)
            W2 = normed_embs[pair[1]].dot(Q)

            # Do 1-cosine to get the actual cosine similarity instead of cosine difference
            pair_results = np.array([1-cosine(W1[i], W2[i]) for i in range(self.num_vertices)])
            results.append(pair_results)

        results = np.asarray(results)
        np.save(os.path.join(save_path, f"{self.file_prefix}_{results_suffix}"), results)
        return np.arange(self.num_vertices), results

    def unaligned_cosine_analysis(self, save_path):
        """ Computes regular cosine similarity values, without alignment, between embedding pairs
        """

        # Set up file naming
        results_suffix = "cossim"

        # Read the embeddings
        normed_embs = {}
        for emb in tqdm(self.embeddings, desc="Reading embeddings"):
            normed_embs[emb] = normalize(
                    self.read_embedding(os.path.join(self.dir, emb))[np.arange(self.num_vertices)], norm='l2',
                    copy=False)

        # Do the analysis
        emb_ind = -1
        results = []
        for pair in tqdm(self.pairs, desc="Comparing embeddings"):

            # only update first embedding if it does not change
            if emb_ind != pair[0]:
                emb_ind = pair[0]

            # Get our two embedding matrics
            W1 = normed_embs[emb_ind]
            W2 = normed_embs[pair[1]]

            # Do 1-cosine to get the actual cosine similarity instead of cosine distance
            pair_results = np.array([1-cosine(W1[i], W2[i]) for i in range(self.num_vertices)])
            results.append(pair_results)

        results = np.asarray(results)
        np.save(os.path.join(save_path, f"{self.file_prefix}_{results_suffix}"), results)
        return np.arange(self.num_vertices), results

    def pairwise_distance(self, nodes, save_path, norm=False):
        """Calculate the pairwise distance between all our embeddings. Save these to the given file path"""

        results_suffix = "euclidean_distance"

        # Read the embeddings
        normed_embs = {}
        for emb in tqdm(self.embeddings, desc="Reading embeddings"):
            # Option for calculating distance between normalized embeddings
            if norm:
              normed_embs[emb] = normalize(
                      self.read_embedding(os.path.join(self.dir, emb))[np.arange(self.num_vertices)], norm='l2',
                      copy=False)
            else:
              normed_embs[emb] = self.read_embedding(os.path.join(self.dir, emb))[np.arange(self.num_vertices)]

        # Do the analysis
        emb_ind = -1
        results = []
        for pair in tqdm(self.pairs, desc="Comparing embeddings"):

            # only update first embedding if it does not change
            if emb_ind != pair[0]:
                emb_ind = pair[0]

            # Get the distance between the two matrices of embeddings and add to results array
            results.append(np.linalg.norm(normed_embs[emb_ind] - normed_embs[pair[1]], axis=1))

        results = np.asarray(results)
        np.save(os.path.join(save_path, f"{self.file_prefix}_{results_suffix}"), results)


    def _get_nodes(self, nodes, num_samples, append):
        """
        Handles getting nodes for the experiments.
        Args:
            nodes: list, node ids
            num_samples: int, how many nodes should be sampled
            append: bool, whether to append sampled nodes to specified nodes
        Returns:
            numpy array of (sampled) node ids
        """
        if nodes is None:
            nodes = self.sample_nodes(num_samples)
        elif append is True:
            # allows specified nodes to be taken twice
            nodes.extend(self.sample_nodes(num_samples))
        return np.asarray(nodes)

    def save_pickle(self, obj, save_path, file_name):
        if save_path is None:
            save_path = os.getcwd()
        if file_name is None:
            # generate name of report from embedding input: use name of first embedding file without number information
            file_name = self.file_prefix
        with open(os.path.join(save_path, f"{file_name}.pickle"), "wb") as f:
            pickle.dump(obj, f)

    def get_combinations(self):
        return self.pairs

    def get_vertex_count(self):
        return self.num_vertices

    # The original code reorders the nodes by ID, but our nodes have the same ordering so there is no need to reorder
    def read_embedding(self, path):
      node_embedding = torch.load(path).detach().numpy()
      return node_embedding

In [7]:
# Code adapted from https://github.com/SGDE2020/embedding_stability/blob/master/similarity_tests/similarity_tests.py
TESTS= ["cossim", "jaccard", "2ndcos", "unalign_cossim", "dist"]

# Expects the the file names in the embedding directory to be of the form model-name_seed_emb.pt
def run_tests(embedding_dir, file_prefix, tests=TESTS, num_nodes=-1, knn_size=20, load_knn=False, kload_size=None,
                results_dir=None, num_processes=4, cossim_sec_indices=None):
    """
    Run specified similarity tests.
    Params:
        embedding_dir: str, path to directory where embeddings are stored
        file_prefix: the prefix for files for saving results
        tests: list of str, tests that will be conducted
        num_nodes: int, number of nodes of a graph that are used in the tests (just the number of nodes in the graph)
        knn_size: int, neighborhood size for knn test
        load_knn: bool, whether a stored knn matrix should be loaded
        nodeinfo_dir = str, path to directory where nodeinfo is stored (tables)
        results_dir = str, path to directory where results will be saved to
    """
    # Store results in the embedding directory in a results file if no other location is specified
    if results_dir is None:
      results_dir = embedding_dir + "results/"
    # Create results directory if it does not exist
    if not os.path.exists(results_dir):
        os.mkdir(results_dir) 

    # Computes a list of all _emb.pt files in a given directory. Assumes the directory being used only contains results from one architecture and data set
    fnames = sorted([f for f in os.listdir(embedding_dir) if f.endswith("_emb.pt")])
    print(fnames)

    if len(fnames) <= 1:
        print(f"Did not find any embeddings for in directory {embedding_dir}")
 
    else:
        # To be compatible with the original source code, we need a list of nodes indices we will be using in our metric computations
        # We use all the nodes
        nodes = [i for i in range(num_nodes)]

        # Allow us to use specific nodes for the second order cossine similarity as it can be very computational expensive
        cossim_nodes = nodes if cossim_sec_indices is None else cossim_sec_indices

        if load_knn and kload_size is None:
            kload_size = knn_size

        # Start tests
        comp = Comparison(emb_dir=embedding_dir, embeddings=fnames, num_nodes=num_nodes, file_prefix=file_prefix)
        if "cossim" in tests:
            print("Executing cosine similarity")
            comp.cossim_analysis(save_path=results_dir)
        if "jaccard" in tests:
            print("Executing jaccard score")
            comp.jaccard_similarity(
                nodes=nodes, append=False, k=knn_size,
                load=load_knn, kload_size=kload_size, save=True, save_path=results_dir, num_processes=num_processes
            )
        if "2ndcos" in tests:
            print("Executing second order cosine similarity")
            comp.second_order_cosine_similarity(
                nodes=cossim_nodes, append=False, k=knn_size,
                save=True, save_path=results_dir, num_processes=num_processes, load=load_knn
            )
        if "unalign_cossim" in tests:
            print("Executing cosine similarity")
            comp.unaligned_cosine_analysis(save_path=results_dir)
        if "dist" in tests:
            print("Executing euclidean distance")
            comp.pairwise_distance(nodes=nodes, save_path=results_dir)



# Run metric calculations

In [11]:
NUM_NODES_CORA = cora_data.x.shape[0]
# CHANGE these to match you directory naming structure
model_directory = "GATV2/"
model_file_prefix = "GATV2_analysis"
# model_directory = "GraphSAGE/"
# model_file_prefix = "GraphSAGE_analysis"

In [13]:
# Run the tests for tests=["cossim", "jaccard"], multiple processes work for these
run_tests(embedding_dir=file_path+model_directory, file_prefix=model_file_prefix, tests=["cossim", "jaccard", "unalign_cossim", "dist"], num_nodes=NUM_NODES_CORA, num_processes=4)

['GATV2_1024258131_emb.pt', 'GATV2_116846604_emb.pt', 'GATV2_1221215631_emb.pt', 'GATV2_1517964140_emb.pt', 'GATV2_170173784_emb.pt', 'GATV2_1818027900_emb.pt', 'GATV2_1860537279_emb.pt', 'GATV2_1863727779_emb.pt', 'GATV2_2036056847_emb.pt', 'GATV2_2105922959_emb.pt', 'GATV2_2135956485_emb.pt', 'GATV2_2342954646_emb.pt', 'GATV2_2455059856_emb.pt', 'GATV2_2739899259_emb.pt', 'GATV2_2794978777_emb.pt', 'GATV2_2952735006_emb.pt', 'GATV2_3300171104_emb.pt', 'GATV2_3303475786_emb.pt', 'GATV2_361232447_emb.pt', 'GATV2_3647665043_emb.pt', 'GATV2_3692371949_emb.pt', 'GATV2_3710910636_emb.pt', 'GATV2_400225693_emb.pt', 'GATV2_4083009686_emb.pt', 'GATV2_4193977854_emb.pt', 'GATV2_516507873_emb.pt', 'GATV2_572297925_emb.pt', 'GATV2_806299656_emb.pt', 'GATV2_880019963_emb.pt', 'GATV2_89475662_emb.pt']
Executing cosine similarity


Comparing embeddings: 100%|██████████| 435/435 [01:33<00:00,  4.63it/s]


Executing jaccard score


Querying nearest neighbors: 100%|██████████| 30/30 [00:23<00:00,  1.29it/s]



 dict_keys(['GATV2_1024258131_emb.pt', 'GATV2_116846604_emb.pt', 'GATV2_1221215631_emb.pt', 'GATV2_1517964140_emb.pt', 'GATV2_170173784_emb.pt', 'GATV2_1818027900_emb.pt', 'GATV2_1860537279_emb.pt', 'GATV2_1863727779_emb.pt', 'GATV2_2036056847_emb.pt', 'GATV2_2105922959_emb.pt', 'GATV2_2135956485_emb.pt', 'GATV2_2342954646_emb.pt', 'GATV2_2455059856_emb.pt', 'GATV2_2739899259_emb.pt', 'GATV2_2794978777_emb.pt', 'GATV2_2952735006_emb.pt', 'GATV2_3300171104_emb.pt', 'GATV2_3303475786_emb.pt', 'GATV2_361232447_emb.pt', 'GATV2_3647665043_emb.pt', 'GATV2_3692371949_emb.pt', 'GATV2_3710910636_emb.pt', 'GATV2_400225693_emb.pt', 'GATV2_4083009686_emb.pt', 'GATV2_4193977854_emb.pt', 'GATV2_516507873_emb.pt', 'GATV2_572297925_emb.pt', 'GATV2_806299656_emb.pt', 'GATV2_880019963_emb.pt', 'GATV2_89475662_emb.pt']) 




100%|██████████| 435/435 [01:45<00:00,  4.12it/s]


Executing cosine similarity


Comparing embeddings: 100%|██████████| 435/435 [01:02<00:00,  7.00it/s]


Executing euclidean distance


Comparing embeddings: 100%|██████████| 435/435 [00:00<00:00, 2968.47it/s]


In [14]:
# Run the tests for tests=["2ndcos"], multiple processes does not work for this so only run on 1, but we can load the nearest neighbours from the previous metric calculations
run_tests(embedding_dir=file_path+model_directory, file_prefix=model_file_prefix, tests=["2ndcos"], num_nodes=NUM_NODES_CORA, num_processes=1, load_knn=True)

['GATV2_1024258131_emb.pt', 'GATV2_116846604_emb.pt', 'GATV2_1221215631_emb.pt', 'GATV2_1517964140_emb.pt', 'GATV2_170173784_emb.pt', 'GATV2_1818027900_emb.pt', 'GATV2_1860537279_emb.pt', 'GATV2_1863727779_emb.pt', 'GATV2_2036056847_emb.pt', 'GATV2_2105922959_emb.pt', 'GATV2_2135956485_emb.pt', 'GATV2_2342954646_emb.pt', 'GATV2_2455059856_emb.pt', 'GATV2_2739899259_emb.pt', 'GATV2_2794978777_emb.pt', 'GATV2_2952735006_emb.pt', 'GATV2_3300171104_emb.pt', 'GATV2_3303475786_emb.pt', 'GATV2_361232447_emb.pt', 'GATV2_3647665043_emb.pt', 'GATV2_3692371949_emb.pt', 'GATV2_3710910636_emb.pt', 'GATV2_400225693_emb.pt', 'GATV2_4083009686_emb.pt', 'GATV2_4193977854_emb.pt', 'GATV2_516507873_emb.pt', 'GATV2_572297925_emb.pt', 'GATV2_806299656_emb.pt', 'GATV2_880019963_emb.pt', 'GATV2_89475662_emb.pt']
Executing second order cosine similarity


Comparing embeddings: 100%|██████████| 435/435 [12:33<00:00,  1.73s/it]


# Compute metric averages

In [22]:
def read_and_average_metric(file_suffix, class_labels, num_classes):
  results_arr = np.load(file_path+model_directory+"/results/"+model_file_prefix+"_"+file_suffix+".npy", mmap_mode='r')
  results_flat = results_arr.flatten() 
  # Compute the 5 number summary for the data across all classes, better than mean and std for potentially skewed distribution
  q1, q2, q3 = np.percentile(results_flat, [25,50,75])
  min, max = results_flat.min(), results_flat.max()
  # Print results
  print("TOTAL STATS")
  print("q1=" + str(q1) + ", q2=" + str(q2) + ", q3=" + str(q3) + ", min=" + str(min) + ", max=" + str(max))

  for i in range(num_classes):
    indices = np.where(class_labels==i)[0]
    # Filter the results by class
    class_vals = results_arr[:, indices]
    # Compute stats for the class
    q1_c, q2_c, q3_c = np.percentile(class_vals, [25,50,75])
    min_c, max_c = class_vals.min(), class_vals.max()
    # Print results
    print("CLASS STATS " + str(i))
    print("q1=" + str(q1_c), ", q2=" + str(q2_c) + ", q3=" + str(q3_c) + ", min=" + str(min_c) + ", max=" + str(max_c))


In [25]:
# Read and compute aligned cosine metric
read_and_average_metric("aligned_cossim", cora_labels, cora_num_classes)

TOTAL STATS
q1=0.9402449727058411, q2=0.9610732793807983, q3=0.974548950791359, min=-0.29593729972839355, max=0.9987619519233704
CLASS STATS 0
q1=0.9330989122390747 , q2=0.9554845094680786, q3=0.969867467880249, min=-0.29593729972839355, max=0.9972626566886902
CLASS STATS 1
q1=0.9277996122837067 , q2=0.9557523131370544, q3=0.9778368473052979, min=0.43271195888519287, max=0.9977697134017944
CLASS STATS 2
q1=0.9412106722593307 , q2=0.9648110568523407, q3=0.9862877577543259, min=0.12200567126274109, max=0.9987534880638123
CLASS STATS 3
q1=0.9457875937223434 , q2=0.9635636806488037, q3=0.97574582695961, min=0.21304240822792053, max=0.9971415996551514
CLASS STATS 4
q1=0.943502813577652 , q2=0.9630200862884521, q3=0.9738742113113403, min=-0.10197541862726212, max=0.9941310286521912
CLASS STATS 5
q1=0.9398431777954102 , q2=0.960607260465622, q3=0.9724498987197876, min=0.33892539143562317, max=0.9987619519233704
CLASS STATS 6
q1=0.9353572726249695 , q2=0.9537501931190491, q3=0.9660337269306183

In [26]:
# Read and compute jaccard metric
read_and_average_metric("20nn_jaccard", cora_labels, cora_num_classes)

TOTAL STATS
q1=0.2903225806451613, q2=0.3793103448275862, q3=0.5384615384615384, min=0.0, max=1.0
CLASS STATS 0
q1=0.2903225806451613 , q2=0.42857142857142855, q3=0.5384615384615384, min=0.0, max=1.0
CLASS STATS 1
q1=0.3333333333333333 , q2=0.42857142857142855, q3=0.6, min=0.0, max=1.0
CLASS STATS 2
q1=0.25 , q2=0.3793103448275862, q3=0.48148148148148145, min=0.0, max=1.0
CLASS STATS 3
q1=0.21212121212121213 , q2=0.3333333333333333, q3=0.48148148148148145, min=0.0, max=1.0
CLASS STATS 4
q1=0.2903225806451613 , q2=0.42857142857142855, q3=0.5384615384615384, min=0.0, max=1.0
CLASS STATS 5
q1=0.2903225806451613 , q2=0.42857142857142855, q3=0.5384615384615384, min=0.0, max=1.0
CLASS STATS 6
q1=0.3793103448275862 , q2=0.48148148148148145, q3=0.6666666666666666, min=0.0, max=1.0


In [27]:
# Read and compute 2nd order cosine similarity metric
read_and_average_metric("20nn_2nd_order_cossim", cora_labels, cora_num_classes)

TOTAL STATS
q1=0.9985876383689604, q2=0.9995815537460984, q3=0.9998620893488287, min=-0.5852828904944805, max=0.9999999999241909
CLASS STATS 0
q1=0.9977243790464189 , q2=0.9992393503863982, q3=0.9997727996096637, min=-0.5852828904944805, max=0.9999999999138507
CLASS STATS 1
q1=0.9972253081187945 , q2=0.999227957990367, q3=0.9999861535347343, min=0.5611754877557246, max=0.9999999999241909
CLASS STATS 2
q1=0.9988987676795811 , q2=0.9996805584075339, q3=0.9999774740652062, min=-0.12874325516282137, max=0.999999999865651
CLASS STATS 3
q1=0.9990104036401778 , q2=0.9996495543902904, q3=0.99985328480206, min=0.0073370611824419285, max=0.9999999998045397
CLASS STATS 4
q1=0.9987578545626989 , q2=0.9995873035764793, q3=0.999842711425067, min=-0.3100520796888264, max=0.99999990637456
CLASS STATS 5
q1=0.9982986896645966 , q2=0.9996013580084357, q3=0.9998574639863003, min=0.5571444246451784, max=0.9999999984800734
CLASS STATS 6
q1=0.9984148304640921 , q2=0.9995803754509792, q3=0.9998375863234535, m

In [28]:
# Read and compute 2nd order cosine similarity metric
read_and_average_metric("cossim", cora_labels, cora_num_classes)

TOTAL STATS
q1=-0.060138775035738945, q2=0.00022699825058225542, q3=0.05984173808246851, min=-0.4605100154876709, max=0.3867574632167816
CLASS STATS 0
q1=-0.05992387607693672 , q2=-0.0011258125305175781, q3=0.05955854803323746, min=-0.4386439919471741, max=0.37736454606056213
CLASS STATS 1
q1=-0.055814918130636215 , q2=0.0020316841546446085, q3=0.06159038096666336, min=-0.33165910840034485, max=0.3131214380264282
CLASS STATS 2
q1=-0.0626735370606184 , q2=-0.0011341748759150505, q3=0.059524315409362316, min=-0.350419819355011, max=0.3724246919155121
CLASS STATS 3
q1=-0.060632698237895966 , q2=-0.000493047758936882, q3=0.057975185103714466, min=-0.4605100154876709, max=0.35503196716308594
CLASS STATS 4
q1=-0.061596255749464035 , q2=0.00041674172098282725, q3=0.06047100480645895, min=-0.38605695962905884, max=0.32571983337402344
CLASS STATS 5
q1=-0.0550233218818903 , q2=0.005741232540458441, q3=0.06467988900840282, min=-0.3515094816684723, max=0.3739445209503174
CLASS STATS 6
q1=-0.062303

In [29]:
read_and_average_metric("euclidean_distance", cora_labels, cora_num_classes)

TOTAL STATS
q1=7.600121021270752, q2=9.494822978973389, q3=11.584370374679565, min=1.4190786, max=22.188509
CLASS STATS 0
q1=7.117758750915527 , q2=8.57570743560791, q3=10.445423126220703, min=1.9378184, max=17.809395
CLASS STATS 1
q1=7.404270887374878 , q2=9.320869445800781, q3=13.247578144073486, min=2.4263067, max=18.194284
CLASS STATS 2
q1=7.964668273925781 , q2=10.090951442718506, q3=13.141571998596191, min=2.4402146, max=18.302643
CLASS STATS 3
q1=7.789259433746338 , q2=9.623552322387695, q3=11.4062659740448, min=2.08738, max=21.272682
CLASS STATS 4
q1=7.7717249393463135 , q2=9.957237720489502, q3=12.058632135391235, min=2.5350184, max=22.188509
CLASS STATS 5
q1=7.646893620491028 , q2=9.437110424041748, q3=11.306429624557495, min=3.4105186, max=18.208906
CLASS STATS 6
q1=7.381295561790466 , q2=9.058689594268799, q3=10.519971132278442, min=1.4190786, max=16.488367


To do:
- Store the nodes being used if we're only using a subset
- Subsets for second_order_cosine_similarity for larger dataset